**ЗАПУСТИ МЕНЯ!** Загрузка стоп-слов, пункутации и т.д. Также открываем доступ к к папке в гугл диске где находится переписки

In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive') #Открываем папку

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Mounted at /content/drive


Алгоритм для проверки частоты и уникальности слова. Этот код как **примерная** демонстрация работы кода в общем.

In [4]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Скачиваем необходимые ресурсы


# Предобработка текста
def preprocess_text(text):
    """
    Очищает текст: убирает пунктуацию, приводит к нижнему регистру, удаляет стоп-слова.
    """
    # Убираем пунктуацию и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    words = word_tokenize(text)
    # Удаляем стоп-слова
    stop_words = set(stopwords.words('russian'))
    return ' '.join([word for word in words if word not in stop_words and len(word) > 2])

# Анализ стиля
def extract_style_keywords(messages, top_n=10):
    """
    Выделяет ключевые слова, характерные для стиля конкретного человека.

    Args:
        messages: Список сообщений человека.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Предобрабатываем все сообщения
    preprocessed_messages = [preprocess_text(msg) for msg in messages]

    # Используем TF-IDF для анализа частотности и уникальности
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=1000)  # Одно- и двусловные фразы
    tfidf_matrix = vectorizer.fit_transform(preprocessed_messages)

    # Получаем слова и их веса
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Средний вес для каждого слова
    keywords = list(zip(feature_names, tfidf_scores))

    # Сортируем по важности
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

    return keywords[:top_n]


# Пример данных
example_chat = {
    "Андрей": [
        "Коль я своиз нашел ёмае, потому как закончим напишу",
        "Кодь я уже в актовом, ты где был",
        "А право или лево",
        "Ёмае",
        "Привет, прости,Ёмае, что долго я думаю примерно 10-20",
        "Ёмае, тут так классно!",
    ],
    "Николай": [
        "Хорошо",
        "Класссс, Андрей! подрапали отсюда",
        "Я на втором ряде с конца",
        "Я сейчас подойду к приёмной комиссии",
        "Насчёт аттестата спрошу",
        "Я уже у футболиста"
    ]
}

# Анализируем стиль Андрея
andrey_keywords = extract_style_keywords(example_chat["Андрей"], top_n=10)
print("Ключевые слова Андрея:", andrey_keywords)

# Анализируем стиль Николая
nikolay_keywords = extract_style_keywords(example_chat["Николай"], top_n=10)
print("Ключевые слова Николая:", nikolay_keywords)


Ключевые слова Андрея: [('ёмае', 0.2755907325005277), ('классно', 0.1058484534202508), ('ёмае классно', 0.1058484534202508), ('актовом', 0.09622504486493762), ('кодь', 0.09622504486493762), ('кодь актовом', 0.09622504486493762), ('лево', 0.09622504486493762), ('право', 0.09622504486493762), ('право лево', 0.09622504486493762), ('закончим', 0.05148523336763505)]
Ключевые слова Николая: [('футболиста', 0.16666666666666666), ('аттестата', 0.07453559924999298), ('аттестата спрошу', 0.07453559924999298), ('втором', 0.07453559924999298), ('втором ряде', 0.07453559924999298), ('комиссии', 0.07453559924999298), ('конца', 0.07453559924999298), ('насчёт', 0.07453559924999298), ('насчёт аттестата', 0.07453559924999298), ('подойду', 0.07453559924999298)]


Вариант-1

In [6]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import download

# Скачиваем необходимые ресурсы для обработки текста


# Получаем список русских стоп-слов и преобразуем в список
russian_stopwords = list(stopwords.words('russian'))

# Функция для предобработки текста
def preprocess_text(text):
    """
    Очищает текст: убирает пунктуацию, приводит к нижнему регистру, удаляет стоп-слова.
    """
    # Убираем пунктуацию и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    words = word_tokenize(text)
    # Удаляем стоп-слова
    custom_stop_words = [
        'это', 'всё', 'тебе', 'просто', 'очень', 'ладно', 'ещё',
        'давай', 'ага', 'так', 'как', 'мне', 'ты', 'он', 'она',
        'мы', 'вы', 'кто', 'где', 'что'
        ]
      # это список стоп слов который мы можем пожеланию дополнять стоп словами, по нашему мнению, точно ничего не характеризующие. Иначе используем стандартный russian_stopwords

    return ' '.join([word for word in words if word not in russian_stopwords and len(word) > 2])

# Функция для выделения ключевых слов
def extract_style_keywords(messages, top_n=10):
    """
    Выделяет ключевые слова, характерные для стиля конкретного человека.

    Args:
        messages: Список сообщений человека.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Предобрабатываем все сообщения
    preprocessed_messages = [preprocess_text(msg) for msg in messages]

    # Используем TF-IDF для анализа частотности и уникальности
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),    # Выделяются фразы от N до M слов (например все фразы имеющие от 1 до 3 слов)
        max_features=50,      # Максимум N фраз
        #ЧЕМ БОЛЬШЕ НЕЗНАЧИТЕЛЬНЫХ ФРАЗ, ТЕМ МЕНЬШЕ ВЫСТАВЛЯЕМ КОЛИЧЕСТВО ВЫДЕЛЯЕМЫХ КЛЮЧ-ФРАЗ
        min_df=100,               # Минимум N данных слов из документа.Чем больше значение, тем меньше выводит редких слов
        #Увеличьте значение, если хотите игнорировать более редкие слова (например, 5 или 10).
        #Уменьшите, если вам нужно включить больше уникальных, но редких слов (например, 1 или 2).
        max_df=0.008,            # Максимум N% документов (от 0 до 100%, включая доли процентов- 0.00001%)
        # Увеличьте значение (например, 0.9), чтобы включить больше слов, если они все еще являются значимыми для анализа.
        # Уменьшите (например, 0.008), если хотите исключить слишком распространенные слова (например, "это", "завтра").
        # ЗНАЧЕНИЕ ВЫШЕ НЕОБХОДИМО МЕНЯТЬ В ЗАВИСИМОСТИ ОТ ВЫВОДА.
        stop_words=russian_stopwords  # Убираем стоп-слова (либо russian_stopwords либо custom_stop_words)
    )
    tfidf_matrix = vectorizer.fit_transform(preprocessed_messages)

    # Получаем слова и их веса
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Средний вес для каждого слова
    keywords = list(zip(feature_names, tfidf_scores))

    # Сортируем по важности
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

    return keywords[:top_n]

# Основной код
def analyze_style(file_path, person_name, top_n=10):
    """
    Анализирует стиль конкретного человека на основе JSON-файла с сообщениями.

    Args:
        file_path: Путь к JSON-файлу.
        person_name: Имя человека, чьи сообщения нужно анализировать.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Загружаем данные из JSON
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Извлекаем сообщения конкретного человека
    messages = []
    for person in data.keys():
        for entry in data.get(person, []):
            if entry['from'] == person_name:
                messages.append(entry['text'])

    if not messages:
        print(f"Нет сообщений для пользователя: {person_name}")
        return []

    # Анализируем стиль и возвращаем ключевые слова
    keywords = extract_style_keywords(messages, top_n=top_n)
    return keywords

# Пример использования
file_path = '/content/drive/MyDrive/StyleMimic/messages.json'  # Укажите путь к JSON-файлу
person_name = 'Никитенко Николай'  # Имя человека, чьи сообщения нужно анализировать

# Получаем ключевые слова
style_keywords = analyze_style(file_path, person_name, top_n=10**9)
print(f"Ключевые слова для {person_name}:")
for word, score in style_keywords:
    print(f"{word}: {score.round(4)}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/StyleMimic/messages.json'

Вариант-2

In [ ]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import download

# Скачиваем необходимые ресурсы для обработки текста
download('punkt')
download('stopwords')

# Получаем список русских стоп-слов и преобразуем в список
russian_stopwords = list(stopwords.words('russian'))

# Функция для предобработки текста
def preprocess_text(text):
    """
    Очищает текст: убирает пунктуацию, приводит к нижнему регистру, удаляет стоп-слова.
    """
    # Убираем пунктуацию и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    words = word_tokenize(text)
    # Удаляем стоп-слова
    custom_stop_words = [
        'это', 'всё', 'тебе', 'просто', 'очень', 'ладно', 'ещё',
        'давай', 'ага', 'так', 'как', 'мне', 'ты', 'он', 'она',
        'мы', 'вы', 'кто', 'где', 'что'
        ]
      # это список стоп слов который мы можем пожеланию дополнять стоп словами, по нашему мнению, точно ничего не характеризующие. Иначе используем стандартный russian_stopwords

    return ' '.join([word for word in words if word not in russian_stopwords and len(word) > 2])


# Функция по удалению дубликатов ключевых слов
def remove_dublicates(keywords):

  sorted_keywords = sorted(keywords, key=lambda x: (-len(x[0].split()), x[1]), reverse=True)# Сортировака по длине фразы
  filtered_keywords = []
  seen_phrases = set()

  for phrase, score in sorted_keywords:
      if not any(phrase in longer for longer in seen_phrases): #Проверка на присутствие дубликата
          filtered_keywords.append((phrase, score))
          seen_phrases.add(phrase)
      # else:
      #     print(phrase)
  return filtered_keywords


# Функция для выделения ключевых слов
def extract_style_keywords(messages, top_n=10):
    """
    Выделяет ключевые слова, характерные для стиля конкретного человека.

    Args:
        messages: Список сообщений человека.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Предобрабатываем все сообщения
    preprocessed_messages = [preprocess_text(msg) for msg in messages]

    # Используем TF-IDF для анализа частотности и уникальности
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),    # Выделяются фразы от N до M слов (например все фразы имеющие от 1 до 3 слов)
        max_features=50,      # Максимум N фраз
        #ЧЕМ БОЛЬШЕ НЕЗНАЧИТЕЛЬНЫХ ФРАЗ, ТЕМ МЕНЬШЕ ВЫСТАВЛЯЕМ КОЛИЧЕСТВО ВЫДЕЛЯЕМЫХ КЛЮЧ-ФРАЗ
        min_df=100,               # Минимум N данных слов из документа.Чем больше значение, тем меньше выводит редких слов
        #Увеличьте значение, если хотите игнорировать более редкие слова (например, 5 или 10).
        #Уменьшите, если вам нужно включить больше уникальных, но редких слов (например, 1 или 2).
        max_df=0.008,            # Максимум N% документов (от 0 до 100%, включая доли процентов- 0.00001%)
        # Увеличьте значение (например, 0.9), чтобы включить больше слов, если они все еще являются значимыми для анализа.
        # Уменьшите (например, 0.008), если хотите исключить слишком распространенные слова (например, "это", "завтра").
        # ЗНАЧЕНИЕ ВЫШЕ НЕОБХОДИМО МЕНЯТЬ В ЗАВИСИМОСТИ ОТ ВЫВОДА.
        stop_words=russian_stopwords  # Убираем стоп-слова (либо russian_stopwords либо custom_stop_words)
    )
    tfidf_matrix = vectorizer.fit_transform(preprocessed_messages)

    # Получаем слова и их веса
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Средний вес для каждого слова
    keywords = list(zip(feature_names, tfidf_scores))

    # Сортируем по важности
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

    keywords = remove_dublicates(keywords)

    return keywords[:top_n]

# Основной код
def analyze_style(file_path, person_name, top_n=10):
    """
    Анализирует стиль конкретного человека на основе JSON-файла с сообщениями.

    Args:
        file_path: Путь к JSON-файлу.
        person_name: Имя человека, чьи сообщения нужно анализировать.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Загружаем данные из JSON
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Извлекаем сообщения конкретного человека
    messages = []
    for person in data.keys():
        for entry in data.get(person, []):
            if entry['from'] == person_name:
                messages.append(entry['text'])

    if not messages:
        print(f"Нет сообщений для пользователя: {person_name}")
        return []

    # Анализируем стиль и возвращаем ключевые слова
    keywords = extract_style_keywords(messages, top_n=top_n)
    return keywords

# Пример использования
file_path = '/content/drive/MyDrive/StyleMimic/messages.json'  # Укажите путь к JSON-файлу
person_name = 'Никитенко Николай'  # Имя человека, чьи сообщения нужно анализировать

# Получаем ключевые слова
style_keywords = analyze_style(file_path, person_name, top_n=10)
print(len(style_keywords))
print(f"Ключевые слова для {person_name}:")
for word, score in style_keywords:
    print(f"{word}: {score.round(4)}")


Вариант-3

In [ ]:
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import download

# Скачиваем необходимые ресурсы для обработки текста
download('punkt')
download('stopwords')

# Получаем список русских стоп-слов и преобразуем в список
russian_stopwords = list(stopwords.words('russian'))

# Функция для предобработки текста
def preprocess_text(text):
    """
    Очищает текст: убирает пунктуацию, приводит к нижнему регистру, удаляет стоп-слова.
    """
    # Убираем пунктуацию и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    words = word_tokenize(text)
    # Удаляем стоп-слова
    custom_stop_words = [
        'это', 'всё', 'тебе', 'просто', 'очень', 'ладно', 'ещё',
        'давай', 'ага', 'так', 'как', 'мне', 'ты', 'он', 'она',
        'мы', 'вы', 'кто', 'где', 'что'
        ]
      # это список стоп слов который мы можем пожеланию дополнять стоп словами, по нашему мнению, точно ничего не характеризующие. Иначе используем стандартный russian_stopwords

    return ' '.join([word for word in words if word not in russian_stopwords and len(word) >= 3])

# Функция для выделения ключевых слов
def extract_style_keywords(messages, top_n=10):
    """
    Выделяет ключевые слова, характерные для стиля конкретного человека.

    Args:
        messages: Список сообщений человека.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Предобрабатываем все сообщения
    preprocessed_messages = [preprocess_text(msg) for msg in messages]

    # Используем TF-IDF для анализа частотности и уникальности
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),    # Выделяются фразы от N до M слов (например все фразы имеющие от 1 до 3 слов)
        max_features=50,      # Максимум N фраз
        #ЧЕМ БОЛЬШЕ НЕЗНАЧИТЕЛЬНЫХ ФРАЗ, ТЕМ МЕНЬШЕ ВЫСТАВЛЯЕМ КОЛИЧЕСТВО ВЫДЕЛЯЕМЫХ КЛЮЧ-ФРАЗ
        min_df=100,               # Минимум N данных слов из документа.Чем больше значение, тем меньше выводит редких слов
        #Увеличьте значение, если хотите игнорировать более редкие слова (например, 5 или 10).
        #Уменьшите, если вам нужно включить больше уникальных, но редких слов (например, 1 или 2).
        max_df=0.008,            # Максимум N% документов (от 0 до 100%, включая доли процентов- 0.00001%)
        # Увеличьте значение (например, 0.9), чтобы включить больше слов, если они все еще являются значимыми для анализа.
        # Уменьшите (например, 0.008), если хотите исключить слишком распространенные слова (например, "это", "завтра").
        # ЗНАЧЕНИЕ ВЫШЕ НЕОБХОДИМО МЕНЯТЬ В ЗАВИСИМОСТИ ОТ ВЫВОДА.
        stop_words=russian_stopwords  # Убираем стоп-слова (либо russian_stopwords либо custom_stop_words)
    )
    tfidf_matrix = vectorizer.fit_transform(preprocessed_messages)

    # Получаем слова и их веса
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.mean(axis=0).A1  # Средний вес для каждого слова
    keywords = list(zip(feature_names, tfidf_scores))

    # Сортируем по важности
    keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

    return keywords[:top_n]

# Основной код
def analyze_style(file_path, person_name, top_n=10):
    """
    Анализирует стиль конкретного человека на основе JSON-файла с сообщениями.

    Args:
        file_path: Путь к JSON-файлу.
        person_name: Имя человека, чьи сообщения нужно анализировать.
        top_n: Количество ключевых слов для извлечения.
    Returns:
        Список ключевых слов с их важностью.
    """
    # Загружаем данные из JSON
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Извлекаем сообщения конкретного человека
    messages = []
    for person in data.keys():
        for entry in data.get(person, []):
          if entry['from'] == person_name and len(entry['text']) <= 300:
              messages.append(entry['text'])

    if not messages:
        print(f"Нет сообщений для пользователя: {person_name}")
        return []

    # Анализируем стиль и возвращаем ключевые слова
    keywords = extract_style_keywords(messages, top_n=top_n)
    return keywords

# Пример использования
file_path = '/content/drive/MyDrive/StyleMimic/messages.json'  # Укажите путь к JSON-файлу
person_name = 'Никитенко Николай'  # Имя человека, чьи сообщения нужно анализировать

# Получаем ключевые слова
style_keywords = analyze_style(file_path, person_name, top_n=500)
print(style_keywords)

# print(f"Ключевые слова для {person_name}:")
# for word, score in style_keywords:
#     print(f"{word}: {score.round(4)}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('вообще', 0.007574188281805199), ('блин', 0.0075383892289878325), ('хотя', 0.007433132726025847), ('хочется', 0.007216718251804946), ('кстати', 0.007003198435915776), ('тобой', 0.006974369091176714), ('понимаю', 0.0068710090934666745), ('привет', 0.006731384314002649), ('милая', 0.006528497653681254), ('сделать', 0.006386087135997872), ('хочешь', 0.0063212866002707365), ('ааа', 0.005946017176986472), ('пока', 0.005666644328296685), ('делать', 0.005580723732083356), ('хотел', 0.005324731259254893), ('думаю', 0.0052694604695762105), ('сказал', 0.005268142455479998), ('очень сильно', 0.005159414706625526), ('сильно люблю', 0.0036808978909536184)]
